In [1]:
import math
import numpy as np
import pandas as pd
import networkx as nx
from functools import partial
from datetime import datetime
from memory_profiler import memory_usage

In [2]:
def euc_2d(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    d = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return d

def generate_nodes(p,data):
    return np.apply_along_axis(lambda x: euc_2d((p[1],p[2]),(x[1],x[2])),1,data)

In [3]:
file = open('./tp2_datasets.txt', 'r')

archives = []

for line in file.readlines()[1:]:
    name = line.split()[0]+".tsp"
    n_nodes = line.split()[1]
    opt = line.split()[2]
    archives.append([name,n_nodes,opt])
file.close()

In [4]:
archive = archives[1]
#for archive in archives:
f = open("./datasets/"+archive[0],"r")
data = f.readlines()[6:-1]
#data.remove("EOF\n")
d = np.array(list(map(lambda x: [float(i) for i in x.split()],data)))
nodes = np.apply_along_axis(lambda x: x,0,d)
edges = np.apply_along_axis(lambda x: generate_nodes(x,nodes),1,nodes)

G = nx.Graph()
#Create Nodes
G.add_nodes_from(nodes[:,0])

for i in range(len(nodes[:,0])):
    for j in range(len(nodes[:,0])):
        G.add_weighted_edges_from([(i+1,j+1,edges[i][j])])

f.close()

# Twice-Around-The-Tree(TAT)

In [5]:
def sum_edges(G,ord):
    c = 0
    for i in range(len(ord)-1):   
        c += G[ord[i]][ord[i+1]]['weight']
    return c + G[ord[0]][ord[len(ord)-1]]['weight']

def twice_around_the_tree(G):
    T = nx.minimum_spanning_tree(G)
    ordenation = list(nx.dfs_preorder_nodes(T, 1))
    total = sum_edges(G, ordenation)
    return total

In [6]:
twice_around_the_tree(G)/float(archive[2])

1.3412907047879643

# Christofides

In [7]:
def christofides(G):

  T = nx.minimum_spanning_tree(G) 
  n_odd = list(filter(lambda x: (x[1]%2)!=0 ,T.degree))
  min_m = nx.min_weight_matching(G.subgraph(n_odd)) 
  h = nx.MultiGraph(T)
  h.add_edges_from(min_m)
  ordenation = list(nx.dfs_preorder_nodes(h, 1))
  total = sum_edges(G, ordenation)
  return total

In [8]:
c = christofides(G)
c/float(archive[2])


1.370741204987188

In [9]:
start=datetime.now()
M_TAT, D_TAT = memory_usage(partial(twice_around_the_tree,G),interval=1,max_usage=True,retval=True)
T_TAT = datetime.now()-start
T_TAT,M_TAT,D_TAT

(datetime.timedelta(seconds=4, microseconds=39643),
 105.6484375,
 10116.014495510826)

In [25]:
from timeout_decorator import timeout

# Define your function
#@timeout(1)  # Set the timeout to 5 seconds
def wrap_up(G):
    start=datetime.now()
    M_TAT, D_TAT =  memory_usage(partial(twice_around_the_tree,G),interval=1,max_usage=True,retval=True)
    T_TAT = datetime.now()-start
    start=datetime.now()
    M_CHF, D_CHF =  memory_usage(partial(christofides,G),interval=1,max_usage=True,retval=True)
    T_CHF = datetime.now()-start
    return [(T_TAT.total_seconds(),M_TAT,D_TAT),(T_CHF.total_seconds(),M_CHF,D_CHF)]

#try:
A = wrap_up(G)
A
#except TimeoutError:
#    print("Function timed out")

[(3.708465, 113.3671875, 10116.014495510826),
 (3.647715, 113.3828125, 10338.130168013373)]

In [26]:
A

[(3.708465, 113.3671875, 10116.014495510826),
 (3.647715, 113.3828125, 10338.130168013373)]

In [14]:
import signal
signal.signal(signal.SIGALRM,lambda x,y:exit())
signal.alarm(30*60)

signal.alarm(0)

AttributeError: module 'signal' has no attribute 'alarm'